In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/latimeshealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,576760256031682561,Sat Mar 14 15:02:15 +0000 2015,Five new running shoes that aim to go the extr...
1,576715414811471872,Sat Mar 14 12:04:04 +0000 2015,Gym Rat: Disq class at Crunch is intense worko...
2,576438353555365888,Fri Mar 13 17:43:07 +0000 2015,Noshing through thousands of ideas at Natural ...
3,576438347003908096,Fri Mar 13 17:43:06 +0000 2015,"Natural Products Expo also explores beauty, su..."
4,576413058177712128,Fri Mar 13 16:02:36 +0000 2015,Free Fitness Weekends in South Bay beach citie...


In [3]:
# -------------------------------------------------
frame.shape

(4171, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Five new running shoes that aim to go the extra mile http://lat.ms/1ELp3wU'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(lambda row: split_txt_form_url(regex_url, row))
frame.head()

http://lat.ms/1ELp3wU
http://lat.ms/1EKOFdr
http://lat.ms/1EHqywg
http://lat.ms/1EHqyfE
http://lat.ms/1EH3SMC
http://lat.ms/1EGNyve
http://lat.ms/1FbQ2kk
http://lat.ms/1b0E0Pi
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1wr2qv0
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1AbI1X4
http://lat.ms/1AbI1X4
http://lat.ms/1wm7sZx
http://lat.ms/1BqdRlG
http://lat.ms/1wm7sZx
http://lat.ms/1AbI1X4
http://lat.ms/1wm7sZx
http://lat.ms/1wm7sZx
http://lat.ms/1A84HY3
http://lat.ms/1A84HY3
http://lat.ms/1Gtx9e7
http://lat.ms/1HbYe32
http://lat.ms/1B9Gsg3
http://lat.ms/1GrwjP1
http://lat.ms/1B9Gsg3
http://lat.ms/1GrngO4
http://lat.ms/1McGw0s
http://lat.ms/1H7OGGg
http://lat.ms/1wKRkBh
http://lat.ms/1wKRkBh
http://lat.ms/1McmTr0
http://lat.ms/1EeQS0o
http://lat.ms/1Kv6yR2
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://lat.ms/17RHiCB
http://lat.ms/1M6N9

http://lati.ms/m70l1
http://lati.ms/m72HF
http://instagram.com/p/aqQAsbhVrR/
http://lat.ms/13LFNye
http://lati.ms/m6ujG
http://lati.ms/m5wR0
http://lati.ms/m5B9s
http://lati.ms/m55Gk
http://lati.ms/m4w9a
http://lati.ms/m32Nb
http://lati.ms/m4vfO
http://instagram.com/p/amTaYdHrGv/
http://lati.ms/m4uXz
http://instagram.com/p/akk1LwPwdd/
http://lati.ms/m3vMW
http://lati.ms/m3pzk
http://lati.ms/m3oQY
http://lati.ms/m36EJ
http://lati.ms/m3vBs
http://lati.ms/m33hr
http://lati.ms/m2V99
http://lati.ms/m3oDu
https://www.strava.com/activities/60257862
http://lati.ms/m2U1N
http://lati.ms/m2DUK
http://lati.ms/m0RYi
http://lati.ms/m1eh0
http://lati.ms/m1f2m
http://lati.ms/m1dGD
http://lati.ms/m19p4
http://lati.ms/m14NY
http://lati.ms/m11fw
http://www.latimes.com/news/science/sciencenow/la-sci-sn-plan-b-contraceptive-fight-leaves-legacy-of-confusion-20130611
http://instagram.com/p/ae-EjLBVhy/
http://lati.ms/m0ILf
http://lati.ms/m0ovM
http://lati.ms/m0ram
http://lati.ms/m0fg6
http://lati.ms/m0c6k
htt

http://lat.ms/yPwYiX
http://lat.ms/wuNWFB
http://lat.ms/xPPchC
http://lat.ms/ArkgmD
http://lat.ms/x43gI5
http://lat.ms/zUFf4G
http://lat.ms/AizPMt
http://lat.ms/wpMg0L
http://lat.ms/yGf0O3
http://lat.ms/w0TrO0
http://lat.ms/wF5EUF
http://lat.ms/wiTsbZ
http://lat.ms/yyErzi
http://lat.ms/AucBJC
http://lat.ms/AuHzoQ
http://lat.ms/zEhrMd
http://lat.ms/zcF04Y
http://lat.ms/wkcc3j
http://lat.ms/zEAmQu
http://lat.ms/x4drxm
http://lat.ms/AsIz4s
http://lat.ms/xw01k1
http://lat.ms/yilRG2
http://lat.ms/zTxBQb
http://lat.ms/xax7Wd
http://lat.ms/ABO88J
http://lat.ms/zFAIZr
http://lat.ms/z9pmuZ
http://lat.ms/z9pmuZ
http://lat.ms/zfklMS
http://lat.ms/yKlJxy
http://lat.ms/zDBVGe
http://lat.ms/zCZKGh
http://lat.ms/yxywHp
http://lat.ms/yqe10C
http://lat.ms/AA2OdS
http://lat.ms/zl8oPp
http://lat.ms/wpe09V
http://lat.ms/zYNRZX
http://lat.ms/Anxevq
http://lat.ms/xCvo4s
http://lat.ms/AzzkKU
http://lat.ms/AroFnm
http://lat.ms/xDrLy8
http://lat.ms/wTwmRZ
http://lat.ms/AcdMGR
http://lat.ms/w708Wp
http://lat.ms

http://lat.ms/1i6m7xX
http://lat.ms/1cPw6mb
http://lat.ms/N1vGlH
http://lat.ms/1oyyOBU
http://lati.ms/umUDr
http://lat.ms/MUUxYA
http://lat.ms/1f9UChT
http://lat.ms/1hP7b7i
http://lat.ms/1f9WQhg
http://lat.ms/1f9UChT
http://lat.ms/1fIIzba
http://lat.ms/1fIIzba
http://lat.ms/1hMOsJL
http://lat.ms/1gRIN2e
http://lat.ms/1c9wQHW
http://lat.ms/1gRIN2e
http://lat.ms/1c9wQHW
http://lat.ms/NRZfqF
http://lat.ms/1gPyagw
http://lat.ms/NRZfqF
http://lat.ms/1c2Ciw3
http://lat.ms/1dV0tb7
http://lat.ms/MJFErW
http://lat.ms/1c2Ciw3
http://lat.ms/1dV0tb7
http://lat.ms/1cadzq9
http://lat.ms/1kavfV1
http://lat.ms/1lm4XPV
http://lat.ms/1gBbh0d
http://lat.ms/1gBbh0d
http://lat.ms/1k87JrP
http://lat.ms/1fDFHB1
http://lat.ms/1c5AgM1
http://lat.ms/1bJELLM
http://lat.ms/1bHOMsU
http://lat.ms/MVCQbV
http://lat.ms/1dp9MzR
http://lat.ms/1pgKIUh
http://ow.ly/tLEfB
http://lat.ms/1h7pDXZ
http://lat.ms/1jmp2lY
http://lat.ms/1jmp2lY
http://lat.ms/1h7pDXZ
http://lati.ms/tSC3W
http://lat.ms/1gTnr6P
http://pbs.twimg.com/

http://lat.ms/UizUbH
http://lat.ms/WnVc2B
http://lat.ms/UcI1GL
http://lat.ms/Wluwzn
http://lat.ms/U163Ey
http://lat.ms/TO7XIw
http://lat.ms/TO3lSL
http://lat.ms/TO3lCh
http://lat.ms/TO3jue
http://lat.ms/TEfxpb
http://lat.ms/W6T1QC
http://lat.ms/TyJG9p
http://lat.ms/W5BPv2
http://lat.ms/Tydwe2
http://lat.ms/W3Lh1P
http://lat.ms/TntAPE
http://lat.ms/TaWker
http://lat.ms/TaWjar
http://lat.ms/TaWin0
http://lat.ms/T8jwtT
http://lat.ms/T8jxOx
http://lat.ms/T6kZB0
http://lat.ms/VUmbT1
http://lat.ms/VTFlbO
http://lat.ms/SV7K5Q
http://lat.ms/SV7HHd
http://lat.ms/SV7Haa
http://lat.ms/SU2IGX
http://lat.ms/SShNsm
http://lat.ms/VPAnwu
http://lat.ms/VOtAmJ
http://lat.ms/SIdQGB
http://lat.ms/VHlSuC
http://lat.ms/VHlPz6
http://lat.ms/VHlPyX
http://lat.ms/Sf42Uh
http://lat.ms/Se59nd
http://lat.ms/VBeroR
http://lat.ms/VAFnVQ
http://lat.ms/VySuXw
http://lat.ms/VyDOYn
http://lat.ms/S6wjfU
http://lat.ms/Vy84me
http://lat.ms/VxvF6G
http://lat.ms/RZWapH
http://lat.ms/RZnEM8
http://lat.ms/Vsf7Nh
http://lat.ms

,tweetid,date,title,url
0,576760256031682561,Sat Mar 14 15:02:15 +0000 2015,Five new running shoes that aim to go the extr...,http://lat.ms/1ELp3wU
1,576715414811471872,Sat Mar 14 12:04:04 +0000 2015,Gym Rat: Disq class at Crunch is intense worko...,http://lat.ms/1EKOFdr
2,576438353555365888,Fri Mar 13 17:43:07 +0000 2015,Noshing through thousands of ideas at Natural ...,http://lat.ms/1EHqywg
3,576438347003908096,Fri Mar 13 17:43:06 +0000 2015,"Natural Products Expo also explores beauty, su...",http://lat.ms/1EHqyfE
4,576413058177712128,Fri Mar 13 16:02:36 +0000 2015,Free Fitness Weekends in South Bay beach citie...,http://lat.ms/1EH3SMC


In [6]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [7]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key,url in enumerate(frame_urls):

    #     url = 'http://bit.ly/VymaIc'
    try:
        # Start the connection
        print(url)
        if len(url):
            if "http://" not in url:
                url = "http://" + url
            res = requests.get(url)

            # Check status header
            if res.status_code == 200:
                print("Header: ", 200)

                # Initialize BeautifulSoup()
                soup = BeautifulSoup(res.content, 'lxml')

                # Get return content as text
                # text_html = res.text

                # Add columns to dataframe
                if title_attempt:
                    df_temp = pd.DataFrame(columns=['author','content','tags'])
                    frame = pd.concat([frame,df_temp])
                    title_attempt = False

                ahref = soup.find('a',{'class':'article_link'})
                if ahref:
                    author = ahref.text            
        #             frame['author'][key] = author[:author.index(",")].lstrip()
                    frame['author'][key] = author.lstrip()
                else:
                    frame['author'] = ''
                frame['content'][key] = soup.find('div',{'class':'article-body'})
            else:
                print("[ERROR]: ", res.status_code)
        else:
            frame['author'][key] = ''
            frame['content'] = ''
        frame['tags'][key] = get_hastags_out(regex_hastags,frame['title'][key])
        frame['title'][key] = stript_hastags(frame['title'][key])
    except (RuntimeError, TypeError, NameError):
        pass

http://lat.ms/1ELp3wU
Header:  200
http://lat.ms/1EKOFdr


scripts.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  def get_hastags_out(regex, sample):
scripts.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
scripts.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
scripts.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-co

Header:  200
http://lat.ms/1EHqywg
Header:  200
http://lat.ms/1EHqyfE
Header:  200
http://lat.ms/1EH3SMC
Header:  200
http://lat.ms/1EGNyve
Header:  200
http://lat.ms/1FbQ2kk
Header:  200

http://lat.ms/1b0E0Pi


scripts.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  def scrapt_w_xpath(xpath_cmd):


Header:  200
http://lat.ms/1KSN5Kh
Header:  200
http://lat.ms/1KSN5Kh
Header:  200
http://lat.ms/1wr2qv0
Header:  200
http://lat.ms/1KSN5Kh
Header:  200
http://lat.ms/1KSN5Kh
Header:  200
http://lat.ms/1KSN5Kh
Header:  200
http://lat.ms/1AbI1X4
Header:  200
http://lat.ms/1AbI1X4
Header:  200
http://lat.ms/1wm7sZx
Header:  200
http://lat.ms/1BqdRlG
Header:  200
http://lat.ms/1wm7sZx
Header:  200
http://lat.ms/1AbI1X4
Header:  200
http://lat.ms/1wm7sZx
Header:  200
http://lat.ms/1wm7sZx
Header:  200
http://lat.ms/1A84HY3
Header:  200
http://lat.ms/1A84HY3
Header:  200
http://lat.ms/1Gtx9e7
Header:  200
http://lat.ms/1HbYe32
Header:  200
http://lat.ms/1B9Gsg3
Header:  200
http://lat.ms/1GrwjP1
Header:  200
http://lat.ms/1B9Gsg3
Header:  200
http://lat.ms/1GrngO4
Header:  200
http://lat.ms/1McGw0s
Header:  200
http://lat.ms/1H7OGGg
Header:  200
http://lat.ms/1wKRkBh
Header:  200
http://lat.ms/1wKRkBh
Header:  200
http://lat.ms/1McmTr0
Header:  200
http://lat.ms/1EeQS0o
Header:  200
http://

Header:  200
http://lat.ms/1IvuoFZ
Header:  200
http://lat.ms/1IvuoFZ
Header:  200
http://lat.ms/1xJvDzw
Header:  200
http://lat.ms/1xJvDzw
Header:  200
http://lat.ms/1AwEYwV
Header:  200
http://lat.ms/1AwFz1S
Header:  200
http://lat.ms/1IvuoFZ
Header:  200
http://lat.ms/1Atz51f
Header:  200
http://lat.ms/1Atz51f
Header:  200
http://lat.ms/1Atz51f
Header:  200
http://lat.ms/1Atz51f
Header:  200
http://lat.ms/13UFer8
Header:  200
http://lat.ms/13UFer8
Header:  200
http://lat.ms/1Aiva9Z
Header:  200
http://lat.ms/1rZnTZE
Header:  200
http://lat.ms/1tKvNau
Header:  200
http://lat.ms/1tKvNau
Header:  200
http://lat.ms/13UFer8
Header:  200
http://lat.ms/1rZnTZE
Header:  200
http://lat.ms/1rZnTZE
Header:  200
http://lat.ms/1tKvNau
Header:  200
http://lat.ms/1tKvNau
Header:  200
http://lat.ms/1A5wCwh
Header:  200
http://lat.ms/1A1IbUZ
Header:  200
http://lat.ms/1AjtNoL
Header:  200
http://lat.ms/1A0E5ws
Header:  200
http://lat.ms/1zR1gJJ
Header:  200
http://lat.ms/1CS6Zjv
Header:  200

http:/

Header:  200
http://lat.ms/10xelbt
Header:  200
http://lat.ms/10xelbt
Header:  200
http://lat.ms/1y5LqpH
Header:  200
http://lat.ms/1xfZEV0
Header:  200
http://lat.ms/1y5LqpH
Header:  200

http://lat.ms/1zwS6RX
Header:  200
http://lat.ms/1pnbtcV
Header:  200
http://lat.ms/1zwS6RX
Header:  200
http://lat.ms/10ozl3I
Header:  200
http://lat.ms/1tc7afx
Header:  200
http://lat.ms/1t8jQDS
Header:  200
http://lat.ms/1zU8opb
Header:  200
http://lat.ms/1ueGEZS
Header:  200
http://lat.ms/1ueBAEZ
Header:  200
http://lat.ms/1ueoZS4
Header:  200
http://lat.ms/1uep1t7
Header:  200
http://ow.ly/DCgis
Header:  200
http://lat.ms/1zPl68L
Header:  200
http://lat.ms/1wK1e3c
Header:  200
http://www.latimes.com/local/california/la-me-ebola-panic-flu-20141025-story.html
Header:  200
http://lat.ms/1tPmGnz
Header:  200
http://lat.ms/1wcQ1Iv
Header:  200
http://lat.ms/1zfBe27
Header:  200
http://lat.ms/12t6BbM
Header:  200
http://lat.ms/12t6xZG
Header:  200
http://lat.ms/1D7yaCy
Header:  200
http://lat.ms/1D7jJ

Header:  200
http://lat.ms/1sueK91
Header:  200
http://lat.ms/1sueK91
Header:  200
http://lat.ms/1oaeyvh


ConnectionError: HTTPConnectionPool(host='overheard.liketodiscover.com', port=80): Max retries exceeded with url: /watch-how-this-supermarket-got-people-to-buy-their-rubbish/?utm_medium=Share_Bottom&utm_source=Facebook-Timeline&utm_content=Photo&utm_campaign=FB_Share (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000013C1349BE48>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [ ]:
# -------------------------------------------------
frame.head()

In [8]:
# -------------------------------------------------
frame.shape

(4171, 7)

In [10]:
frame.to_csv('./latimes_scrapt.csv', sep='|', encoding='utf-8')